In [51]:
import os

In [52]:
%pwd

'/Users/ujjwaltiwari/Desktop/projects/kidney-disease-classifier-ct-scan'

In [3]:
os.chdir('../')

In [53]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/UjjTiw/kidney-disease-classifier-ct-scan.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'UjjTiw'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'aed5cdc8e6331b6bacc6a684372ea07f6d913399'

In [54]:
import tensorflow as tf

In [55]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int


In [56]:
from DiseaseClassifier.constants import  *
from DiseaseClassifier.utils.common import read_yaml, create_directories, save_json

In [57]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model= 'artifacts/training/model.h5',
            training_data= 'artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone',
            mlflow_uri= 'https://dagshub.com/UjjTiw/kidney-disease-classifier-ct-scan.mlflow',
            all_params= self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
        )
        return eval_config
    

In [68]:
import tensorflow as tf
from pathlib import Path
import mlflow
from urllib.parse import urlparse

In [73]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            # if tracking_url_type_store != "file":

            #     # Register the model
            #     # There are other ways to use the Model Registry, which depends on the use case,
            #     # please refer to the doc for more information:
            #     # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            #     mlflow.tensorflow.log_model(self.model, "model", registered_model_name="EfficientNetB7")
            # else:
            #     mlflow.tensorflow.log_model(self.model, "model")

In [74]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation  = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-04-14 12:13:28,960: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-14 12:13:28,970: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-14 12:13:28,972: INFO: common: Created directory at : artifacts]
Found 3732 images belonging to 4 classes.
117/117 [==============================] - 416s 4s/step - loss: 118.1015 - accuracy: 0.4767
[2024-04-14 12:20:33,212: INFO: common: json files saved at: scores.json]


In [ ]:
hmm = tf.keras.models.load_model('artifacts/training/model.h5')
type(hmm)

keras.src.engine.functional.Functional

In [67]:
print(tf.__version__)

2.14.0


In [75]:
print(tf.keras.__version__)

AttributeError: module 'tensorflow.keras' has no attribute '__version__'